In [1]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from transformers import DistilBertModel, DistilBertTokenizer
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import sys
from model import DistilBertClassifier
from dataset import NewsDataset
from sklearn import metrics

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
is_colab = 'google.colab' in sys.modules
is_train = False

In [3]:
if is_colab:
    from google.colab import drive
    drive.mount('./mnt')
    base_dir = './mnt/My Drive/coursework'
else:
    base_dir = '.'

In [4]:
dataframe = pd.read_csv(f'{base_dir}/data/findata.csv', encoding='cp1252', header=None)
dataframe.columns = ['sentiment', 'title']
if is_train:
    tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', truncation=True, do_lower_case=True)
else:
    tokenizer = DistilBertTokenizer.from_pretrained(f'{base_dir}/savepoints/vocab.pt', truncation=True, do_lower_case=True)

In [5]:
train_df, test_df = train_test_split(dataframe, test_size=.2)
train_ds = NewsDataset(train_df, tokenizer, 128)
test_ds = NewsDataset(test_df, tokenizer, 128)

train_loader = DataLoader(train_ds, 16, shuffle=True)
test_loader = DataLoader(test_ds, 16, shuffle=True)

In [6]:
model = DistilBertClassifier().to(device)
model

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


DistilBertClassifier(
  (l1): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Line

In [7]:
if is_train:
    loss_fn = nn.BCEWithLogitsLoss()
    optim = torch.optim.Adam(model.parameters(), lr=2e-5)

    for epoch in range(1, 3):
        print(f'epoch {epoch:1}')
        epoch_loss = 0
        for bn, data in tqdm(enumerate(train_loader)):
            out = model(data['ids'].to(device), data['mask'].to(device), data['token_type_ids'].to(device))
            optim.zero_grad()
            
            loss = loss_fn(out, data['targets'].to(device))
            epoch_loss = loss.item()

            loss.backward()
            optim.step()
            if bn%50 == 0:
                print(f'\tloss: {epoch_loss/(bn+1):.5f}')
        print(f'epoch {epoch:1} loss {epoch_loss/bn:.5f}')
        torch.save(model.state_dict(), f'{base_dir}/savepoints/model.pt')
else:
    model.load_state_dict(torch.load(f'{base_dir}/savepoints/model.pt', map_location=device))

In [13]:
tokens = tokenizer.encode_plus(
            'microsoft stock is doing bad',
            None,
            add_special_tokens=True,
            max_length=128,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
out = model(torch.tensor(tokens['input_ids']).view(1, -1).to(device), torch.tensor(tokens['attention_mask']).view(1, -1).to(device), None)
out

tensor([[-4.9914, -5.4532,  5.0071]], grad_fn=<AddmmBackward0>)

In [37]:
model.eval()
fin_targets=np.array([]).reshape(0, 3)
fin_outputs=np.array([]).reshape(0, 3)
with torch.no_grad():
    for _, data in tqdm(enumerate(test_loader)):
        ids = data['ids'].to(device)
        mask = data['mask'].to(device)
        token_type_ids = data['token_type_ids'].to(device)
        targets = data['targets'].to(device)
        outputs = model(ids, mask, token_type_ids)
        fin_targets = np.concatenate((fin_targets, targets.cpu().detach().numpy()))
        fin_outputs = np.concatenate((fin_outputs, outputs.cpu().detach().numpy()))

61it [00:56,  1.09it/s]


In [41]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [42]:
metrics.hamming_loss(fin_targets, sigmoid(fin_outputs) >= .5)

0.03230240549828179

In [51]:
scores = {
    'r2': lambda y_true, y_pred: metrics.r2_score(y_true, y_pred),
    'f1': lambda y_true, y_pred: metrics.f1_score(np.argmax(y_true, axis=1), np.argmax(y_pred, axis=1), average=None),
    'accuracy': lambda y_true, y_pred: metrics.accuracy_score(np.argmax(y_true, axis=1), np.argmax(y_pred, axis=1)),
}

In [52]:
scores['accuracy'](fin_targets, sigmoid(fin_outputs))

0.9515463917525773

In [45]:
np.argmax(fin_targets[:10], axis=1)

array([0, 0, 1, 2, 1, 1, 0, 1, 0, 1])

In [46]:
fin_targets[:10]

array([[1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.]])